In [ ]:
import time
import websocket
import rel
import _thread

from binance.client import Client

import pandas as pd
import numpy as np

import warnings
warnings.filterwarnings("ignore")

from talib import abstract

In [ ]:
with open('../data/api.txt', 'r') as file:
    api_key = file.readlines()[0].split("\n")[0]

In [ ]:
with open('../data/api.txt', 'r') as file:
    api_secret = file.readlines()[1].split("\n")[0]

In [ ]:
client = Client(api_key, api_secret)

In [ ]:
symbol = "ETHUSDT" #in uppercase
frequency = "4h" #in lowercase except month
endpoint = f"wss://stream.binance.com:9443/ws/{symbol}@kline_{frequency}"

In [ ]:
#parameters for obtaining historical data

klines_dict = {
    
    "1m" : Client.KLINE_INTERVAL_1MINUTE,
    "3m" : Client.KLINE_INTERVAL_3MINUTE,
    "5m" : Client.KLINE_INTERVAL_5MINUTE,
    "15m" : Client.KLINE_INTERVAL_15MINUTE,
    "30m" : Client.KLINE_INTERVAL_30MINUTE,
    "1h" : Client.KLINE_INTERVAL_1HOUR,
    "2h" : Client.KLINE_INTERVAL_2HOUR,
    "4h" : Client.KLINE_INTERVAL_4HOUR,
    "6h" : Client.KLINE_INTERVAL_6HOUR,
    "8h" : Client.KLINE_INTERVAL_8HOUR,
    "12h" : Client.KLINE_INTERVAL_12HOUR,
    "1d" : Client.KLINE_INTERVAL_1DAY,
    "3d" : Client.KLINE_INTERVAL_3DAY,
    "1w" : Client.KLINE_INTERVAL_1WEEK,
    "1M" : Client.KLINE_INTERVAL_1MONTH

}

#how far back you want historical data? examples:

# 30 minutes ago UTC
# 1 day ago UTC
# 1 Dec, 2017

In [ ]:
klines = client.get_historical_klines(symbol, klines_dict[frequency], "1 Dec, 2017")

In [ ]:
open_time, open_, high, low, close, volume, close_time, volume_quoted, num_trades = ([] for i in range(9))

for candle in klines:
    open_time.append(candle[0])
    open_.append(candle[1])
    high.append(candle[2])
    low.append(candle[3])
    close.append(candle[4])
    volume.append(candle[5])
    close_time.append(candle[6])
    volume_quoted.append(candle[7])
    num_trades.append(candle[8])

In [ ]:
data = pd.DataFrame({"Date" : open_time, "Open" : open_, "High" : high, "Low" : low, "Close" : close, "Volume" : volume_quoted})

In [ ]:
data.Date = pd.to_datetime(data.Date, unit='ms')

data.set_index(data.Date, inplace=True, drop=True)

data.drop("Date", axis=1, inplace=True)

for column in data.columns:
    data[column] = data[column].astype(float)
    
data.index.name = None

In [ ]:
# for talib to calculate indicators
inputs = {
    'open': np.array(data.Open),
    'high': np.array(data.High),
    'low': np.array(data.Low),
    'close': np.array(data.Close),
    'volume': np.array(data.Volume)
}

In [ ]:
print(f"Number of rows: {len(data)}")
print(f"Starting date: {data.index[0]}")

In [ ]:
#indicators
data["EMA200"] = abstract.EMA(inputs, timeperiod=200)
data["Slope200"] = abstract.LINEARREG_SLOPE(inputs, timeperiod=200)
data["EMA21"] = abstract.EMA(inputs, timeperiod=21)
data["Slope21"] = abstract.LINEARREG_SLOPE(inputs, timeperiod=21)
data["EMA13"] = abstract.EMA(inputs, timeperiod=13)
data["Slope13"] = abstract.LINEARREG_SLOPE(inputs, timeperiod=13)
data["RSI"] = abstract.RSI(inputs, timeperiod=14)

In [ ]:
data.dropna(inplace=True) #drop rows with null values

In [ ]:
from backtesting import Backtest, Strategy
from backtesting.lib import crossover, TrailingStrategy, cross

In [ ]:
class MyStrategy(Strategy):
    
    risk = 0.015
    stop_loss = 38
    stop_loss = stop_loss/1000
    a = 124 # slope 200 EMA for longs
    c = 85 # take profit for shorts
    s_200 = -3 #slope for 200 EMA for shorts
    s_21 = 292
    s_13 = -16
    rsi = 53
    
    def init(self):
        
        super().init()
    
    def next(self):
        
        super().next()
                
        if len(self.trades) > 0 and self.position.is_long:
            if self.data.EMA200 > self.data.EMA13:
                self.position.close()
        elif len(self.trades) > 0 and self.position.is_short:
            if self.data.EMA200 < self.data.EMA13:
                self.position.close()
             
        elif crossover(self.data.EMA13, self.data.EMA200) and len(self.trades) == 0 and self.data.Slope200 < self.a/100:
            self.buy(size=self.risk/self.stop_loss, sl=self.data.Close*(1-self.stop_loss))
            
        elif cross(self.data.EMA13, self.data.EMA200) and len(self.trades) == 0 and self.data.EMA13 < self.data.EMA200:
            self.sell(size=self.risk/self.stop_loss, sl=self.data.Close*(1+self.stop_loss), tp=self.data.Close*(self.c/100))
            
        elif self.data.Close < self.data.EMA200 and self.data.Slope200 < (self.s_200/100) and self.data.Slope21 < (self.s_21/100) and self.data.Slope13 < (self.s_13/100) and len(self.trades) == 0 and self.data.RSI > self.rsi:
            self.sell(size=self.risk/self.stop_loss, sl=self.data.Close*(1+self.stop_loss), tp=self.data.Close*(self.c/100))
            
                
bt = Backtest(data, MyStrategy, cash=25000, commission=0.001)
stats = bt.run()
stats

In [ ]:
trades = stats._trades # results

In [ ]:
len(trades[trades.Size < 0]) #short trades

In [ ]:
trades[trades.Size<0].Duration.mean() #average short duration

In [ ]:
len(trades[(trades.Size < 0) & (trades.ReturnPct > 0)]) / len(trades[trades.Size < 0]) * 100 # win ratio shorts %

In [ ]:
len(trades[trades.Size > 0]) #long trades

In [ ]:
trades[trades.Size>0].Duration.mean() #average long duration

In [ ]:
len(trades[(trades.Size > 0) & (trades.ReturnPct > 0)]) / len(trades[trades.Size > 0])*100 # win ratio longs %

In [ ]:
(1+stats._trades[:21].ReturnPct).cumprod().iloc[-1] # cumulative returns 2018 (-88.31%) 21 trades

In [ ]:
(1+stats._trades[21:39].ReturnPct).cumprod().iloc[-1] # cumulative returns 2019 (1,74%) 18 trades

In [ ]:
(1+stats._trades[39:54].ReturnPct).cumprod().iloc[-1] # cumulative returns 2020 (470,16%) 15 trades

In [ ]:
(1+stats._trades[54:71].ReturnPct).cumprod().iloc[-1] # cumulative returns 2021 (399,20%) 17 trades

In [ ]:
(1+stats._trades[71:].ReturnPct).cumprod().iloc[-1] # cumulative returns now 2022 (now -63.85%) 26 trades

In [ ]:
0.975**27 #28 trades to loose half your money

In [ ]:
(1 + trades[88:].ReturnPct).cumprod().iloc[-1] 
#profitability during bootcamp period (since July) 37.5% win rate 9 trades 3 wins 5 losses 1 neutral

## Visualization

In [ ]:
filtered = trades[["Size","EntryBar","ExitBar"]]
short_entries = list(filtered[filtered.Size<0].EntryBar)
short_closes = list(filtered[filtered.Size<0].ExitBar)
long_entries = list(filtered[filtered.Size>0].EntryBar)
long_closes = list(filtered[filtered.Size>0].ExitBar)

In [ ]:
data["Entry_trade"] = [np.nan] * len(data)
data["Closing_trade"] = [np.nan] * len(data)

In [ ]:
for num, row in enumerate(data["Closing_trade"]):
    if num in long_closes:
        data["Closing_trade"].iloc[num] = data["Close"].iloc[num]+60 # long closing code
    elif num in short_closes:
        data["Closing_trade"].iloc[num] = data["Close"].iloc[num]-60 # short closing code

In [ ]:
for num, row in enumerate(data["Entry_trade"]):
    if num in long_entries:
        data["Entry_trade"].iloc[num] = data["Close"].iloc[num]+60 # long entry code
    elif num in short_entries:
        data["Entry_trade"].iloc[num] = data["Close"].iloc[num]-60 # short entry code

In [ ]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from datetime import datetime

In [ ]:
data_plot = data[9823:] #prework and bootcamp duration (change to desired length)

figure = go.Figure(data= [go.Candlestick(x=data_plot.index,
                                       open = data_plot.Open,
                                       high = data_plot.High,
                                       low = data_plot.Low,
                                       close = data_plot.Close,
                                       increasing_line_color= 'yellow', decreasing_line_color= 'blue'),
                                       go.Scatter(x=data_plot.index ,y=data_plot.EMA200 , line=dict(color='purple', width=1), name="EMA200"),
                                       go.Scatter(x=data_plot.index ,y=data_plot.EMA21 , line=dict(color='pink', width=1), name="EMA21"),
                                       go.Scatter(x=data_plot.index ,y=data_plot.EMA13 , line=dict(color='black', width=1), name="EMA13")])

figure.add_scatter(x=data_plot.index, y=data_plot['Entry_trade'], mode="markers",
                marker=dict(size=10, color="Green"),
                name="EntrySignals")

figure.add_scatter(x=data_plot.index, y=data_plot['Closing_trade'], mode="markers",
                marker=dict(size=6, color="Red"),
                name="ClosingSignals")

figure.update(layout_xaxis_rangeslider_visible = False) #to hide slider
figure.update_layout(margin_l = 0, margin_b = 0, margin_r = 0, margin_t = 50) # for margins
# figure.update_xaxis(showline=True, linewidth=2, linecolor="black", gridcolor="black")
# figure.update_yaxis(showline=True, linewidth=2, linecolor="black", gridcolor="black")
figure.update_layout(title='The Ethereum Bot 4H Strategy', yaxis_title='ETH Price')

#increasing_line_color= 'cyan', decreasing_line_color= 'gray' to change candle colors (include in data list)
figure.show()

In [ ]:
import plotly.express as px # to export figure

figure.write_html("ethereum4hbotnew.html")